<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-13 09:52:08
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.37 C
-------------------
Today PnL: -2.39 L (-1.72%)
Current PnL: -32.70 L (-20.85%)
CY Booked + Current PnL: -17.96 L (-11.45%)
-------------------
Total profit:  1.25 L
Total loss:  -33.95 L
-------------------
Total Booked + Current PnL: 9.02 L (6.95%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.78%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 98.06 L (71.68%)
Deployed:  1.30 C
Current:  1.37 C
CAGR/XIRR %: 2.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-0.01,2.88,18.17,21.57,18315.0,2820.0,100800.0,397.04,-11.47,61.0,H-SC,3.65,168.0,0.15,0.74,47.10,XR,ATH,FINANCE
18,COALINDIA,-1.38,1.65,17.29,19.23,26443.0,2480.0,152940.0,484.83,19.88,43.0,L-LC,11.83,182.0,0.09,1.12,23.27,XY25,ATH,MINING
77,TTKPRESTIG,-1.96,-26.41,36.01,0.09,26707.0,-26611.0,74166.0,770.00,72.04,41.0,M-SC,8.76,253.0,-1.00,0.54,2.07,OX40N,NTT,DURABLES
36,ICICIGI,-0.64,1.04,18.11,19.34,36618.0,2086.0,202195.0,2252.93,-20.26,55.0,X-MC,3.29,71.0,0.06,1.48,17.15,X40,ATH,INSURANCE
51,MEDANTA,-1.62,-5.65,31.50,24.08,37731.0,-7170.0,119780.0,1486.00,-13.35,50.0,X-SC,11.76,91.0,-0.19,0.88,9.71,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,6.43,-43.22,138.24,35.27,89129.0,-49075.0,64474.0,80.00,-55.73,56.0,L-SC,14.33,268.0,-0.55,0.47,57.80,XR,NTT,HOTELS
11,BANDHANBNK,0.76,-18.44,135.93,92.42,308811.0,-51375.0,227184.0,400.00,59.48,74.0,H-SC,8.33,169.0,-0.17,1.66,31.29,XY24,NTT,BANKS
43,IRCTC,0.56,-24.61,102.86,52.93,209761.0,-66579.0,203929.0,1269.00,-44.20,46.0,H-MC,12.71,120.0,-0.32,1.49,2.97,XY24,BTT,TRAVEL
50,MASFIN,-0.01,2.88,18.17,21.57,18315.0,2820.0,100800.0,397.04,-11.47,61.0,H-SC,3.65,168.0,0.15,0.74,47.10,XR,ATH,FINANCE
44,ITC,-0.33,-22.45,42.86,10.78,79874.0,-53958.0,186360.0,452.00,-53.82,39.0,X-LC,18.92,1.0,-0.68,1.36,2.65,X40,NTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BSOFT,-11.31,-30.24,116.14,50.79,113961.0,-42529.0,98124.0,831.70,-10.32,38.0,H-SC,7.66,171.0,-0.37,0.72,14.25,XR,ATH,IT
42,INFY,-6.16,-11.46,51.62,34.24,143002.0,-35860.0,277028.0,1972.00,-31.08,16.0,X-LC,4.70,6.0,-0.25,2.03,0.00,X40,NTT,IT
73,TCS,-4.76,-27.99,64.62,18.55,176016.0,-105851.0,272386.0,4311.59,-37.86,23.0,X-LC,2.47,3.0,-0.60,1.99,0.00,X40,ATH,IT
70,SYMPHONY,-4.43,-31.65,46.26,-0.03,54113.0,-54165.0,116976.0,1306.00,-45.13,46.0,M-SC,10.35,206.0,-1.00,0.86,8.10,OX40N,NTT,DURABLES
33,HCLTECH,-4.42,-8.44,35.26,23.83,78099.0,-20428.0,221496.0,1908.19,-4.20,19.0,X-LC,5.16,13.0,-0.26,1.62,6.27,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-1.53,2.15,103.53,107.91,156149.0,3181.0,150825.0,1641.55,-16.90,62.0,M-SC,10.01,220.0,0.02,1.10,13.47,OX40N,ATH,CABLES
66,SIS,-2.89,-24.10,59.76,21.26,50365.0,-26754.0,84278.0,528.00,1988.08,48.0,H-SC,4.03,163.0,-0.53,0.62,16.19,OX40N,NTT,MISC
77,TTKPRESTIG,-1.96,-26.41,36.01,0.09,26707.0,-26611.0,74166.0,770.00,72.04,41.0,M-SC,8.76,253.0,-1.00,0.54,2.07,OX40N,NTT,DURABLES
40,INDIGOPNTS,-2.47,-30.90,44.74,0.02,53969.0,-53932.0,120627.0,1408.00,84.15,28.0,M-SC,7.45,240.0,-1.00,0.88,4.92,OX40N,NTT,PAINTS
72,TATAELXSI,-3.85,-31.05,91.47,32.03,83152.0,-40929.0,90906.0,9161.00,-25.44,31.0,H-SC,6.20,157.0,-0.49,0.66,1.82,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-1.38,1.65,17.29,19.23,26443.0,2480.0,152940.0,484.83,19.88,43.0,L-LC,11.83,182.0,0.09,1.12,23.27,XY25,ATH,MINING
50,MASFIN,-0.01,2.88,18.17,21.57,18315.0,2820.0,100800.0,397.04,-11.47,61.0,H-SC,3.65,168.0,0.15,0.74,47.10,XR,ATH,FINANCE
26,FINCABLES,-1.53,2.15,103.53,107.91,156149.0,3181.0,150825.0,1641.55,-16.90,62.0,M-SC,10.01,220.0,0.02,1.10,13.47,OX40N,ATH,CABLES
1,ABB,-0.46,5.72,37.70,45.57,104269.0,14957.0,276576.0,7934.00,-32.08,67.0,H-MC,2.98,121.0,0.14,2.02,22.82,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-0.01,2.88,18.17,21.57,18315.0,2820.0,100800.0,397.04,-11.47,61.0,H-SC,3.65,168.0,0.15,0.74,47.10,XR,ATH,FINANCE
1,ABB,-0.46,5.72,37.70,45.57,104269.0,14957.0,276576.0,7934.00,-32.08,67.0,H-MC,2.98,121.0,0.14,2.02,22.82,AR,NTT,ELECTRICAL
26,FINCABLES,-1.53,2.15,103.53,107.91,156149.0,3181.0,150825.0,1641.55,-16.90,62.0,M-SC,10.01,220.0,0.02,1.10,13.47,OX40N,ATH,CABLES
39,INDIAMART,-0.62,-6.04,120.00,106.72,139068.0,-7446.0,115890.0,4810.62,-53.49,45.0,H-SC,7.59,138.0,-0.05,0.85,15.93,AR,ATH,MISC
85,ZYDUSLIFE,-1.60,-4.44,41.93,35.63,83973.0,-9304.0,200270.0,1286.17,-18.07,52.0,H-MC,5.96,119.0,-0.11,1.46,11.49,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-6.16,-11.46,51.62,34.24,143002.0,-35860.0,277028.0,1972.00,-31.08,16.0,X-LC,4.70,6.0,-0.25,2.03,0.00,X40,NTT,IT
33,HCLTECH,-4.42,-8.44,35.26,23.83,78099.0,-20428.0,221496.0,1908.19,-4.20,19.0,X-LC,5.16,13.0,-0.26,1.62,6.27,X40,ATH,IT
73,TCS,-4.76,-27.99,64.62,18.55,176016.0,-105851.0,272386.0,4311.59,-37.86,23.0,X-LC,2.47,3.0,-0.60,1.99,0.00,X40,ATH,IT
8,AWL,-2.52,-34.82,141.16,57.19,300068.0,-113554.0,212573.0,485.00,-69.24,25.0,X-MC,17.42,58.0,-0.38,1.55,0.00,XY24,NTT,FMCG
13,BERGEPAINT,-0.85,-18.61,48.85,21.14,90384.0,-42315.0,185024.0,680.00,-24.35,27.0,X-MC,11.09,75.0,-0.47,1.35,0.00,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,-0.78,0.79,42.58,43.70,106458.0,1950.0,250018.0,735.00,-7.59,53.0,X-MC,1.83,73.0,0.02,1.83,15.95,XY24,BTT,FMCG
81,VBL,-1.29,-9.14,49.32,35.67,141535.0,-28870.0,286972.0,671.64,-19.99,38.0,X-LC,2.44,5.0,-0.20,2.10,3.45,X40N,ATH,FMCG
73,TCS,-4.76,-27.99,64.62,18.55,176016.0,-105851.0,272386.0,4311.59,-37.86,23.0,X-LC,2.47,3.0,-0.60,1.99,0.00,X40,ATH,IT
78,UNITDSPR,-0.65,3.45,16.74,20.77,40546.0,8066.0,242210.0,1644.00,-5.50,61.0,X-MC,2.57,70.0,0.20,1.77,11.10,X40N,NTT,BREWERIES
34,HINDUNILVR,-2.09,-8.22,23.85,13.67,56272.0,-21123.0,235940.0,2922.00,-15.11,46.0,X-LC,2.68,9.0,-0.38,1.73,11.38,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
2,ABBOTINDIA,-0.94,-11.40,33.26,18.07,52704.0,-20395.0,158460.0,35195.00,-24.11,35.0,X-MC,10.14,64.0,-0.39,1.16,0.0,X40,ATH,PHARMA
8,AWL,-2.52,-34.82,141.16,57.19,300068.0,-113554.0,212573.0,485.00,-69.24,25.0,X-MC,17.42,58.0,-0.38,1.55,0.0,XY24,NTT,FMCG
13,BERGEPAINT,-0.85,-18.61,48.85,21.14,90384.0,-42315.0,185024.0,680.00,-24.35,27.0,X-MC,11.09,75.0,-0.47,1.35,0.0,XY24,NTT,PAINTS
42,INFY,-6.16,-11.46,51.62,34.24,143002.0,-35860.0,277028.0,1972.00,-31.08,16.0,X-LC,4.70,6.0,-0.25,2.03,0.0,X40,NTT,IT
53,PGHH,-1.27,-13.56,52.73,32.01,97318.0,-28960.0,184560.0,17616.87,-39.22,33.0,X-MC,9.22,60.0,-0.30,1.35,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-2.29,-32.05,110.21,42.84,48682.0,-20834.0,44172.0,424.00,-55.37,45.0,X-SC,20.07,83.0,-0.43,0.32,6.25,XY24,NTT,MISC
58,RELAXO,-2.37,-53.65,228.58,52.31,153800.0,-77875.0,67285.0,1176.00,-50.63,35.0,X-SC,13.85,92.0,-0.51,0.49,0.06,X40N,NTT,FOOTWEAR
12,BATAINDIA,-1.48,-43.83,140.70,35.20,101692.0,-56394.0,72276.0,2096.00,-5.27,43.0,X-SC,20.96,93.0,-0.55,0.53,2.71,X40,NTT,FOOTWEAR
51,MEDANTA,-1.62,-5.65,31.50,24.08,37731.0,-7170.0,119780.0,1486.00,-13.35,50.0,X-SC,11.76,91.0,-0.19,0.88,9.71,XY24,NTT,HEALTHCARE
35,HONAUT,-1.55,-22.96,86.03,43.32,107950.0,-37392.0,125480.0,58357.33,-33.85,38.0,X-SC,10.28,90.0,-0.35,0.92,1.88,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.33,-22.45,42.86,10.78,79874.0,-53958.0,186360.0,452.00,-53.82,39.0,X-LC,18.92,1.0,-0.68,1.36,2.65,X40,NTT,FMCG
75,TMPV,-1.66,-22.71,59.11,22.98,103204.0,-51301.0,174597.0,600.00,-19.41,60.0,X-LC,3.37,2.0,-0.50,1.28,11.62,XY24,NTT,AUTO
73,TCS,-4.76,-27.99,64.62,18.55,176016.0,-105851.0,272386.0,4311.59,-37.86,23.0,X-LC,2.47,3.0,-0.60,1.99,0.00,X40,ATH,IT
81,VBL,-1.29,-9.14,49.32,35.67,141535.0,-28870.0,286972.0,671.64,-19.99,38.0,X-LC,2.44,5.0,-0.20,2.10,3.45,X40N,ATH,FMCG
42,INFY,-6.16,-11.46,51.62,34.24,143002.0,-35860.0,277028.0,1972.00,-31.08,16.0,X-LC,4.70,6.0,-0.25,2.03,0.00,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-1.43,-32.85,97.91,32.89,50759.0,-25364.0,51842.0,1800.00,-740.36,51.0,L-MC,11.10,259.0,-0.50,0.38,42.77,XR,NTT,BANKS
67,SONACOMS,-1.58,-9.28,53.32,39.09,48932.0,-9389.0,91770.0,804.02,-29.03,61.0,M-SC,4.40,227.0,-0.19,0.67,29.82,AR,ATH,AUTO
50,MASFIN,-0.01,2.88,18.17,21.57,18315.0,2820.0,100800.0,397.04,-11.47,61.0,H-SC,3.65,168.0,0.15,0.74,47.10,XR,ATH,FINANCE
79,VAIBHAVGBL,-0.46,-21.10,103.76,60.77,149631.0,-38566.0,144209.0,521.00,66.39,61.0,H-SC,1.39,158.0,-0.26,1.05,30.74,XR,NTT,JEWELLERY
18,COALINDIA,-1.38,1.65,17.29,19.23,26443.0,2480.0,152940.0,484.83,19.88,43.0,L-LC,11.83,182.0,0.09,1.12,23.27,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,0.76,-18.44,135.93,92.42,308811.0,-51375.0,227184.0,400.00,59.48,74.0,H-SC,8.33,169.0,-0.17,1.66,31.29,XY24,NTT,BANKS
79,VAIBHAVGBL,-0.46,-21.10,103.76,60.77,149631.0,-38566.0,144209.0,521.00,66.39,61.0,H-SC,1.39,158.0,-0.26,1.05,30.74,XR,NTT,JEWELLERY
1,ABB,-0.46,5.72,37.70,45.57,104269.0,14957.0,276576.0,7934.00,-32.08,67.0,H-MC,2.98,121.0,0.14,2.02,22.82,AR,NTT,ELECTRICAL
67,SONACOMS,-1.58,-9.28,53.32,39.09,48932.0,-9389.0,91770.0,804.02,-29.03,61.0,M-SC,4.40,227.0,-0.19,0.67,29.82,AR,ATH,AUTO
7,ATULAUTO,-2.62,-13.96,69.19,45.57,117352.0,-27523.0,169609.0,844.00,3678.38,66.0,M-SC,5.84,245.0,-0.23,1.24,28.35,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.16
1,25,44.63
2,50,76.65


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.74
MC    31.22
LC    23.05
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.80
X40      22.94
X40N     15.32
AR        8.98
XR        8.85
XY25      8.80
OX40N     7.43
SR        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.49
H-SC    22.95
X-LC    19.73
M-SC    11.57
X-SC    10.41
H-MC     4.97
M-MC     1.38
H-LC     1.13
L-LC     1.12
M-LC     1.07
L-SC     0.81
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.07,-14.99,53.63
FINANCE,13.48,-13.29,57.87
IT,11.32,-38.15,107.08
MISC,7.11,-33.73,88.43
ELECTRICAL,6.20,-10.57,50.91
PAINTS,5.07,-33.07,52.84
INSURANCE,4.90,-1.33,36.01
PHARMA,4.15,-3.43,35.98
AUTO,3.19,-20.23,61.81


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3303704.0,21
X40,1360027.0,15
AR,1342336.0,10
XR,1280283.0,12
X40N,1027318.0,10
OX40N,776033.0,10
XY25,417719.0,6
SR,298552.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3657172.0,24
X-MC,1677778.0,16
M-SC,1473996.0,15
X-LC,1193635.0,12
X-SC,868720.0,9
H-MC,398003.0,3
L-SC,179228.0,2
M-LC,145111.0,1
H-LC,78358.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1247326.0      6
           AR         952598.0      5
M-SC       XY24       828336.0      6
H-SC       XR         814102.0      7
X-LC       X40        631520.0      6
X-MC       X40        553943.0      7
           XY24       496910.0      3
           X40N       415359.0      4
X-SC       X40N       381501.0      4
H-SC       OX40N      344594.0      4
M-SC       OX40N      341340.0      5
X-SC       XY24       312655.0      3
H-SC       SR         298552.0      2
X-LC       X40N       230458.0      2
X-MC       XY25       211566.0      2
H-MC       XY24       209761.0      1
X-LC       XY24       208716.0      2
H-MC       AR         188242.0      2
M-SC       XR         181182.0      2
X-SC       X40        174564.0      2
M-LC       XR         145111.0      1
M-SC       AR         123138.0      2
X-LC       XY25       122941.0      2
L-SC       OX40N       90099.0      1
           XR          89129.0      1
H-LC       AR          78358.0      1
M-MC       XY25        56769.0      1
L-MC       XR          50759.0      1
L-LC       XY25        26443.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
